The purpose of this file is to allow us to go step by step on how all functions work.
We also want to provide the opportunitiy to identify any issues within each function. 
Also, we would like to test how long each function takes to run. 

The main function that we will be working with is known as the "process_it" function. 

This function does the following things: 

1. If there are any county values that we want to filter for, it will clean these up. 

By this, we mean that it standardize how each county is presented in the column. 
One thing to note is that in Louisiana counties are actually known as parishes. 
this is important because there are a lot of emitters in Louisiana. 

2. Next, the function inserts a location column 

3. Next, we filter the data by referring back to [1] if its length is positive. 

4. Next, we iterate through the filtered dataframe and inser any missing info. 

Mainly, we are grabbing unit type, parent company and fuel type. 

5. Next, we save the excel. 

6. Lastly, if user wants we then create a map that plots each facility 

We need to consider created a heatmap as well as it can be helpful. 

First, let's load some packages that we will be using. 

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import functions as fs
import requests
from xml.etree import ElementTree as ET
import xml
from uszipcode import SearchEngine
import folium

AttributeError: module 'sqlalchemy_mate' has no attribute 'ExtendedBase'

Next, let's load our data as a pandas dataframe.

In [8]:
# we will be loading 2022 emissions data
df = pd.read_excel('ghgp_data_files/ghgp_data_2022.xlsx',
                    sheet_name='Direct Emitters', skiprows=3)

# let's see what our data looks like
print(df.describe())
print(df.info())

        Facility Id        FRS Id      Zip Code     Latitude    Longitude  \
count  6.491000e+03  6.325000e+03   6491.000000  6491.000000  6491.000000   
mean   1.005753e+06  1.100347e+11  54205.365121    37.259143   -92.450338   
std    4.026099e+03  2.896094e+07  26985.992487     5.990902    16.973396   
min    1.000001e+06  1.100003e+11      0.000000    13.297100  -174.113611   
25%    1.002432e+06  1.100024e+11  32533.000000    32.797318   -97.857184   
50%    1.005123e+06  1.100332e+11  55337.000000    37.757930   -90.263967   
75%    1.007824e+06  1.100701e+11  77507.000000    41.233660   -82.651856   
max    1.014729e+06  1.100714e+11  99801.000000    71.292071   144.891130   

       Primary NAICS Code  Total reported direct emissions  \
count         6367.000000                     6.491000e+03   
mean        367896.649285                     3.857296e+05   
std         144283.115806                     1.119423e+06   
min         111419.000000                     0.000000e+00

Okay, now let's remove columns that we really don't need given they don't have
enough data or don't provide much insight. 

In [9]:
# now we want to also only select the columns that we are interested in
keep_cols = ['Facility Id', 'Facility Name', 'City', 'State', 'Zip Code', 'Address', 
                'County', 'Latitude', 'Longitude', 'Primary NAICS Code', 'Industry Type (subparts)', 'Industry Type (sectors)', 
                'Total reported direct emissions', 'CO2 emissions (non-biogenic) ', 'Methane (CH4) emissions ', 'Stationary Combustion', 
                'Electricity Generation']

# now let's filter our data
df = df[keep_cols]

# now we want to rename the following columns to names that we can easily refer # to
new_cols = ['facility_id', 'facility_name', 'city', 'state', 'zipcode', 'address', 
            'county', 'latitude', 'longitude', 'primary_naics_code', 'idustry_type_subparts', 'industry_type_sectors', 'total_reported_e', 'co2_e', 'methane_e', 'stationary_comb_e', 'electricty_gen_e',]

# now we want to rename our columns to make more sense and be more accessible
df.columns = new_cols  

print(df.describe())
print(df.info())

        facility_id       zipcode     latitude    longitude  \
count  6.491000e+03   6491.000000  6491.000000  6491.000000   
mean   1.005753e+06  54205.365121    37.259143   -92.450338   
std    4.026099e+03  26985.992487     5.990902    16.973396   
min    1.000001e+06      0.000000    13.297100  -174.113611   
25%    1.002432e+06  32533.000000    32.797318   -97.857184   
50%    1.005123e+06  55337.000000    37.757930   -90.263967   
75%    1.007824e+06  77507.000000    41.233660   -82.651856   
max    1.014729e+06  99801.000000    71.292071   144.891130   

       primary_naics_code  total_reported_e         co2_e     methane_e  \
count         6367.000000      6.491000e+03  5.841000e+03  6.383000e+03   
mean        367896.649285      3.857296e+05  4.012173e+05  2.013747e+04   
std         144283.115806      1.119423e+06  1.164798e+06  8.041914e+04   
min         111419.000000      0.000000e+00  0.000000e+00  2.500000e-01   
25%         221112.000000      3.320953e+04  2.543350e+04

Okay, now let's clean our county values.

In [10]:
def clean_county_val(dataframe):
    '''
    The purpose of this function is to iterate through each row in our dfs and remove the word County from the County column value
    '''

    # these are the words that we want to remove
    words_to_remove = ['County', 'county', 'COUNTY', 'Parish', 'parish', 
                       'PARISH']

    # regular expression pattern to match any of the words 
    pattern = "|".join(words_to_remove)

    # removing the words from the column value 
    dataframe['county'] = dataframe['county'].str.replace(pattern, '', regex=True)

    # making all County names uppercase
    dataframe['county'] = dataframe['county'].str.upper()

    return dataframe

In [11]:
df = clean_county_val(dataframe=df)

print(df.describe())
print(df.info())

        facility_id       zipcode     latitude    longitude  \
count  6.491000e+03   6491.000000  6491.000000  6491.000000   
mean   1.005753e+06  54205.365121    37.259143   -92.450338   
std    4.026099e+03  26985.992487     5.990902    16.973396   
min    1.000001e+06      0.000000    13.297100  -174.113611   
25%    1.002432e+06  32533.000000    32.797318   -97.857184   
50%    1.005123e+06  55337.000000    37.757930   -90.263967   
75%    1.007824e+06  77507.000000    41.233660   -82.651856   
max    1.014729e+06  99801.000000    71.292071   144.891130   

       primary_naics_code  total_reported_e         co2_e     methane_e  \
count         6367.000000      6.491000e+03  5.841000e+03  6.383000e+03   
mean        367896.649285      3.857296e+05  4.012173e+05  2.013747e+04   
std         144283.115806      1.119423e+06  1.164798e+06  8.041914e+04   
min         111419.000000      0.000000e+00  0.000000e+00  2.500000e-01   
25%         221112.000000      3.320953e+04  2.543350e+04

Now, let's insert locations dependent on zipcode.

In [14]:
def zip_to_county(zipcode):

    '''
    The purpose of this file is to return a coutny when given a zipcode value. 

    Specifically, we will be working with an EPA Emissions data file for 2021 that provides us with emissions data for the united states. 

    The main issue is that the data is incomplete in terms of county values. 
    Thus, we need to go into each worksheet and fill in the county values by using the reported zipcodes that are included...
    '''

    search = SearchEngine()
    result = search.by_zipcode(zipcode)
    
    if result:
        return result.county
        
    else:
        return None


def insert_location(dataframe):

    '''
    The purpose of this function is to iterate through each row of a dataframe and add a county value if there is not one already there
    '''

    for index, row in dataframe.iterrows():

        # check if County column is empty 
        county_value = row['county']

        if pd.isna(county_value) or county_value == '':

            # 
            dataframe.at[index, 'county'] = zip_to_county(row['zipcode'])

        else:

            # if not empty, then just leave alone
            continue

    # now let's add a new column to our dataframe that adds the County and the State
    dataframe['location'] = dataframe['county'] + ', ' + dataframe['state']

    return dataframe

In [15]:
df = insert_location(dataframe=df)

print(df.describe())
print(df.info())

        facility_id       zipcode     latitude    longitude  \
count  6.491000e+03   6491.000000  6491.000000  6491.000000   
mean   1.005753e+06  54205.365121    37.259143   -92.450338   
std    4.026099e+03  26985.992487     5.990902    16.973396   
min    1.000001e+06      0.000000    13.297100  -174.113611   
25%    1.002432e+06  32533.000000    32.797318   -97.857184   
50%    1.005123e+06  55337.000000    37.757930   -90.263967   
75%    1.007824e+06  77507.000000    41.233660   -82.651856   
max    1.014729e+06  99801.000000    71.292071   144.891130   

       primary_naics_code  total_reported_e         co2_e     methane_e  \
count         6367.000000      6.491000e+03  5.841000e+03  6.383000e+03   
mean        367896.649285      3.857296e+05  4.012173e+05  2.013747e+04   
std         144283.115806      1.119423e+06  1.164798e+06  8.041914e+04   
min         111419.000000      0.000000e+00  0.000000e+00  2.500000e-01   
25%         221112.000000      3.320953e+04  2.543350e+04

Now, we want to go add 3 new cols of data that need to be grabbed from another
EPA related website. 
The key thing we need here is the ghgp id or the facility id.

First, let's load into our environment the meander_xml code that achieves this

In [16]:
def meander_xml(ghgp_id, topic, year=2022):

    '''
    Given a ghgp_id, return the fuel type within the xml code.
    The purpose of this function is extract and process an XML file.

    Not entirely sure what an XML file is, or how it compares to a JSON file but it seems
    very low memory and has a tree structure.

    This is how facilities provide reports to the EPA emissions entity. 

    Please note that you need to:

    'pip install elementpath' and then import the module into your env

    to import: 
    'xml.etree.ElementTree as ET'
    '''

    try:

        # here, we will be adding the ghgp_id value to a url used to make GET request
        url = f'https://ghgdata.epa.gov/ghgp/service/xml/{year}?id={ghgp_id}&et=undefined'

        # making the url GET request for the xml file
        response = requests.get(url=url)
        webpage_content = response.text

        # modifying our url contnet because it containts code we do not need
        lines = webpage_content[38:-6].split('\n')
        modified_content = '\n'.join(lines[1:])

        # parse the XML
        root = ET.fromstring(modified_content)

        # now we want to create an ifelse statement that will grab specific data
        # user needs

        if topic == 'Fuel Type' or topic == 'fuel type':

            # creating a list to return the found elements to
            fuel_types = []

            # extract the fuel type element
            for fuel_type in root.findall(".//FuelType"):

                # return the fuel_type value
                fuel_types.append(fuel_type.text)
            
            # removing redundancies
            fuel_types = list(set(fuel_types))

            # concatenating our list to output a long string
            fuel_typez = ", ".join([str(item) for item in fuel_types])

            return fuel_typez
    

        elif topic == 'Unit Type' or topic == 'unit type':
    
            # creating a list to return the found elements to
            unit_types = []

            # extract the unit type value
            for unit_type in root.findall('.//UnitType'):

                # return the unit_type value
                unit_types.append(unit_type.text)
            
            # removing redundancies
            unit_types = list(set(unit_types))

            # concatenating our list to output a long string
            unit_typez = ", ".join([str(item) for item in unit_types])

            return unit_typez
    

        elif topic =='Parent Company' or topic == 'parent company': 
        
            # creating a list to return the found elements to
            parent_cos = []
            
            # else, extract parent company name 
            for parent_co in root.findall('.//ParentCompanyLegalName'):

                # return the parent company name
                parent_cos.append(parent_co.text)
            
            # removing redundancies
            parent_cos = list(set(parent_cos))

            # concatenating our list to output a long string
            parent_coz = ", ".join([str(item) for item in parent_cos])

            return parent_coz
    

        else: 

            # means none of the 3 topics were given 
            print('Error: please insert a valid topic to receive a valid value!')


    except xml.etree.ElementTree.ParseError:
        # xml file is not properly formatted, so skip this iteration
        "sorry, bud!"
        pass
    

We have now loaded the code and now need to run it through every row in our df

In [ ]:
    # adding the needed empty columns in our dataframe
    # these columns will be populated with data from meander_xml function
    df['Unit Type'] = ''
    df['Parent Company'] = ''
    df['Fuel Type'] = ''   


    for index, row in df.iterrows():

        '''
        purpose is to go through, grab the 3 values and add them to the dataframe
        '''

        # defining our ghgp_id from the row we just grabbed
        ghgp = row['facility_id']
        print(ghgp)

        # now we will be grabbing and adding the 3 values to our datframe
        # adding a unit type value
        df.loc[index, 'Unit Type'] = meander_xml(ghgp_id=ghgp, topic='unit type')

        # adding a parent company value
        df.loc[index, 'Parent Company'] = meander_xml(ghgp_id=ghgp, topic='parent company')

        # adding a fuel type value
        df.loc[index, 'Fuel Type'] = meander_xml(ghgp_id=ghgp, topic='fuel type')